# YaResNet.

> Yet Another ResNet model.

In [1]:
#hide
import torch
import torch.nn as nn

from nbdev.showdoc import show_doc
from IPython.display import Markdown, display

In [2]:
# hide
def print_doc(func_name):
    doc = show_doc(func_name, title_level=4, disp=False)
    display(Markdown(doc))

In [3]:
#hide
# from nbdev.showdoc import *
# from fastcore.test import *
# import torch
# import torch.nn as nn
# from functools import partial
# from collections import OrderedDict
# from model_constructor.layers import SEBlock, ConvLayer, act_fn, noop, SimpleSelfAttention
# from model_constructor.net import Net
# from model_constructor.activations import Mish

## YaResBlock

In [4]:
#hide
from model_constructor.yaresnet import YaResBlock

In [5]:
#hide_input
print_doc(YaResBlock)

<h4 id="YaResBlock" class="doc_header"><code>class</code> <code>YaResBlock</code><a href="https://github.com/ayasyrev/model_constructor/tree/master/model_constructor/yaresnet.py#L15" class="source_link" style="float:right">[source]</a></h4>

> <code>YaResBlock</code>(**`expansion`**, **`ni`**, **`nh`**, **`stride`**=*`1`*, **`conv_layer`**=*`ConvLayer`*, **`act_fn`**=*`ReLU(inplace=True)`*, **`zero_bn`**=*`True`*, **`bn_1st`**=*`True`*, **`pool`**=*`AvgPool2d(kernel_size=2, stride=2, padding=0)`*, **`sa`**=*`False`*, **`sym`**=*`False`*, **`groups`**=*`1`*, **`dw`**=*`False`*, **`div_groups`**=*`None`*, **`se_module`**=*`SEModule`*, **`se`**=*`False`*, **`se_reduction`**=*`16`*) :: `Module`

YaResBlock. Reduce by pool instead of stride 2

In [6]:
#collapse_output
bl = YaResBlock(1,64,64,sa=True)
bl

YaResBlock(
  (convs): Sequential(
    (conv_0): ConvLayer(
      (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act_fn): ReLU(inplace=True)
    )
    (conv_1): ConvLayer(
      (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (sa): SimpleSelfAttention(
      (conv): Conv1d(64, 64, kernel_size=(1,), stride=(1,), bias=False)
    )
  )
  (merge): ReLU(inplace=True)
)

In [7]:
#hide
bs_test = 16
xb = torch.randn(bs_test, 64, 32, 32)
y = bl(xb)
print(y.shape)
assert y.shape == torch.Size([bs_test, 64, 32, 32]), f"size"

torch.Size([16, 64, 32, 32])


In [8]:
#collapse_output
bl = YaResBlock(1,64,64,se=True)
bl

YaResBlock(
  (convs): Sequential(
    (conv_0): ConvLayer(
      (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act_fn): ReLU(inplace=True)
    )
    (conv_1): ConvLayer(
      (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (se): SEModule(
      (squeeze): AdaptiveAvgPool2d(output_size=1)
      (excitation): Sequential(
        (fc_reduce): Linear(in_features=64, out_features=4, bias=True)
        (se_act): ReLU(inplace=True)
        (fc_expand): Linear(in_features=4, out_features=64, bias=True)
        (se_gate): Sigmoid()
      )
    )
  )
  (merge): ReLU(inplace=True)
)

In [9]:
#hide
bs_test = 16
xb = torch.randn(bs_test, 64, 32, 32)
y = bl(xb)
print(y.shape)
assert y.shape == torch.Size([bs_test, 64, 32, 32]), f"size"

torch.Size([16, 64, 32, 32])


In [10]:
#collapse_output
bl = YaResBlock(4,64,128,stride=2,act_fn=nn.LeakyReLU(), bn_1st=False)
bl

YaResBlock(
  (reduce): AvgPool2d(kernel_size=2, stride=2, padding=0)
  (convs): Sequential(
    (conv_0): ConvLayer(
      (conv): Conv2d(256, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (act_fn): LeakyReLU(negative_slope=0.01)
      (bn): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (conv_1): ConvLayer(
      (conv): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (act_fn): LeakyReLU(negative_slope=0.01)
      (bn): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (conv_2): ConvLayer(
      (conv): Conv2d(128, 512, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
  )
  (idconv): ConvLayer(
    (conv): Conv2d(256, 512, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=Tru

In [11]:
#hide
bs_test = 16
xb = torch.randn(bs_test, 256, 32, 32)
y = bl(xb)
print(y.shape)
assert y.shape == torch.Size([bs_test, 512, 16, 16]), f"size"

torch.Size([16, 512, 16, 16])


In [12]:
#collapse_output
bl = YaResBlock(4,64,128,stride=2,act_fn=nn.LeakyReLU(), bn_1st=False, groups=4)
bl

YaResBlock(
  (reduce): AvgPool2d(kernel_size=2, stride=2, padding=0)
  (convs): Sequential(
    (conv_0): ConvLayer(
      (conv): Conv2d(256, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (act_fn): LeakyReLU(negative_slope=0.01)
      (bn): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (conv_1): ConvLayer(
      (conv): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=4, bias=False)
      (act_fn): LeakyReLU(negative_slope=0.01)
      (bn): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (conv_2): ConvLayer(
      (conv): Conv2d(128, 512, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
  )
  (idconv): ConvLayer(
    (conv): Conv2d(256, 512, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running

In [13]:
#hide
bs_test = 16
xb = torch.randn(bs_test, 256, 32, 32)
y = bl(xb)
print(y.shape)
assert y.shape == torch.Size([bs_test, 512, 16, 16]), f"size"

torch.Size([16, 512, 16, 16])


In [14]:
#collapse_output
bl = YaResBlock(4,64,128,stride=2,act_fn=nn.LeakyReLU(), bn_1st=False, div_groups=4)
bl

YaResBlock(
  (reduce): AvgPool2d(kernel_size=2, stride=2, padding=0)
  (convs): Sequential(
    (conv_0): ConvLayer(
      (conv): Conv2d(256, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (act_fn): LeakyReLU(negative_slope=0.01)
      (bn): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (conv_1): ConvLayer(
      (conv): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
      (act_fn): LeakyReLU(negative_slope=0.01)
      (bn): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (conv_2): ConvLayer(
      (conv): Conv2d(128, 512, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
  )
  (idconv): ConvLayer(
    (conv): Conv2d(256, 512, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_runnin

In [15]:
#hide
bs_test = 16
xb = torch.randn(bs_test, 256, 32, 32)
y = bl(xb)
print(y.shape)
assert y.shape == torch.Size([bs_test, 512, 16, 16]), f"size"

torch.Size([16, 512, 16, 16])


In [16]:
#collapse_output
bl = YaResBlock(4,64,128,stride=2,act_fn=nn.LeakyReLU(), bn_1st=False, dw=True)
bl

YaResBlock(
  (reduce): AvgPool2d(kernel_size=2, stride=2, padding=0)
  (convs): Sequential(
    (conv_0): ConvLayer(
      (conv): Conv2d(256, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (act_fn): LeakyReLU(negative_slope=0.01)
      (bn): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (conv_1): ConvLayer(
      (conv): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=128, bias=False)
      (act_fn): LeakyReLU(negative_slope=0.01)
      (bn): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (conv_2): ConvLayer(
      (conv): Conv2d(128, 512, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
  )
  (idconv): ConvLayer(
    (conv): Conv2d(256, 512, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_runni

In [17]:
#hide
bs_test = 16
xb = torch.randn(bs_test, 256, 32, 32)
y = bl(xb)
print(y.shape)
assert y.shape == torch.Size([bs_test, 512, 16, 16]), f"size"

torch.Size([16, 512, 16, 16])


## YaResNet constructor.

In [19]:
from model_constructor import ModelConstructor
yaresnet  = ModelConstructor(block=YaResBlock, stem_sizes = [3, 32, 64, 64], name='YaResNet')

In [20]:
yaresnet

YaResNet constructor
  c_in: 3, c_out: 1000
  expansion: 1, groups: 1, dw: False, div_groups: None
  sa: False, se: False
  stem sizes: [3, 32, 64, 64], stride on 0
  body sizes [64, 128, 256, 512]
  layers: [2, 2, 2, 2]

In [21]:
yaresnet.block_sizes, yaresnet.layers

([64, 64, 128, 256, 512], [2, 2, 2, 2])

In [22]:
#collapse_output
yaresnet.stem

Sequential(
  (conv_0): ConvLayer(
    (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (act_fn): ReLU(inplace=True)
  )
  (conv_1): ConvLayer(
    (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (act_fn): ReLU(inplace=True)
  )
  (conv_2): ConvLayer(
    (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (act_fn): ReLU(inplace=True)
  )
  (stem_pool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
)

In [23]:
#hide
bs_test = 16
xb = torch.randn(bs_test, 3, 128, 128)
y = yaresnet.stem(xb)
print(y.shape)
assert y.shape == torch.Size([bs_test, 64, 32, 32]), f"size"

torch.Size([16, 64, 32, 32])


/home/jzz/anaconda3/envs/pt_19/lib/python3.8/site-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /opt/conda/conda-bld/pytorch_1623448278899/work/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


In [24]:
#collapse_output
yaresnet.body

Sequential(
  (l_0): Sequential(
    (bl_0): YaResBlock(
      (convs): Sequential(
        (conv_0): ConvLayer(
          (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (act_fn): ReLU(inplace=True)
        )
        (conv_1): ConvLayer(
          (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        )
      )
      (merge): ReLU(inplace=True)
    )
    (bl_1): YaResBlock(
      (convs): Sequential(
        (conv_0): ConvLayer(
          (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (act_fn): ReLU(inplace=True)
        )
        (conv_1): ConvLayer(
          (conv):

In [25]:
#collapse_output
yaresnet.head

Sequential(
  (pool): AdaptiveAvgPool2d(output_size=1)
  (flat): Flatten()
  (fc): Linear(in_features=512, out_features=1000, bias=True)
)

Lots of experiments showed that it worth trying Mish activation function.

In [27]:
#collapse_output
yaresnet.act_fn = torch.nn.Mish()
yaresnet()

Sequential(
  YaResNet
  (stem): Sequential(
    (conv_0): ConvLayer(
      (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act_fn): Mish()
    )
    (conv_1): ConvLayer(
      (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act_fn): Mish()
    )
    (conv_2): ConvLayer(
      (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act_fn): Mish()
    )
    (stem_pool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  )
  (body): Sequential(
    (l_0): Sequential(
      (bl_0): YaResBlock(
        (convs): Sequential(
          (conv_0): ConvLayer(
            (co

In [30]:
#hide
bs_test = 16
xb = torch.randn(bs_test, 3, 128, 128)
y = yaresnet()(xb)
print(y.shape)
assert y.shape == torch.Size([bs_test, 1000]), f"size"

torch.Size([16, 1000])


# end
model_constructor
by ayasyrev